### Qgis, centerline add length, convert to point ,add index
```
Algorithm 'Field calculator' starting…
Input parameters:
{ 'FIELD_LENGTH' : 10, 'FIELD_NAME' : 'length', 'FIELD_PRECISION' : 3, 'FIELD_TYPE' : 0, 'FORMULA' : '$length', 'INPUT' : 'G:/810_change_field/linestrings_single_clip.shp', 'OUTPUT' : 'G:/810_change_field/linestrings_single_clip_index_length.shp' }

Algorithm 'Extract vertices' starting…
Input parameters:
{ 'INPUT' : 'G:/810_change_field/linestrings_single_clip_index_length.shp', 'OUTPUT' : 'G:/810_change_field/linestrings_single_clip_index_length_vertice.shp' }

Algorithm 'Field calculator' starting…
Input parameters:
{ 'FIELD_LENGTH' : 10, 'FIELD_NAME' : 'index', 'FIELD_PRECISION' : 3, 'FIELD_TYPE' : 1, 'FORMULA' : '@row_number', 'INPUT' : 'G:/810_change_field/linestrings_single_clip_index_length_vertice.shp', 'OUTPUT' : 'G:/810_change_field/linestrings_single_clip_index_length_vertice_index.shp' }

```


### Qgis,centerline get width, export to csv

```
Algorithm 'Distance to nearest hub (line to hub)' starting…
Input parameters:
{ 'FIELD' : 'id', 'HUBS' : 'G:/811/batiment_single_polygon_clip_index_boundary_densify3_point.shp', 'INPUT' : 'G:/810_change_field/linestrings_single_clip_index_length_vertice_index.shp', 'OUTPUT' : 'G:/811/centerline_width.shp', 'UNIT' : 0 }
```

### etree csv to xml

In [1]:
import csv
import xml.etree.ElementTree as ET

In [2]:
root = ET.Element ('network')
root.set('version','1.0')
nodes=ET.SubElement(root, 'nodes')
links=ET.SubElement(root, 'links')

In [3]:
dict_hub = {}

In [4]:
with open ("centerline_width.csv",'rt',encoding='utf-8') as f:
    current_group = None
    current_hub=None
    reader = csv.reader(f)
    next(reader, None)
    for row in reader :
        WKT, FID,length, vertex_ind, vertex_par, vertex_p_1, distance, angle, index,HubName,HubDist = row
        
        #single_node
        ET.SubElement(nodes, 'node',{'nodeId': index, 'geometry':WKT[19:-2].split(',')[0], 'linkId':FID, 'vertex_ind':vertex_ind})
        #links
        ##id length
        if current_group is None or FID != current_group.get('linkId'):
            current_group = ET.SubElement(links, 'link',{'linkId':FID,'length':length })
        
        ## nodes
        if (vertex_ind=='0'):
            attributes = ET.SubElement(current_group,'attributes')
            attribute = ET.SubElement(attributes, 'attribute',{'Type': 'Topology','name':'nodes','nodeId':index})   
        else: 
            list_nodeId = attribute.get('nodeId')
            attribute.set('nodeId',list_nodeId+" "+index)
        ## width
        if FID not in dict_hub.keys():
            dict_hubDist = {}
            dict_hubDist [HubName] = HubDist
            dict_hub[FID]= dict_hubDist
        else:
            if (HubName in dict_hub[FID].keys()):
                dict_hub[FID][HubName]= dict_hub[FID][HubName] + " "+ HubDist
            else:
                dict_hub[FID][HubName]= HubDist
            
        

In [5]:
##set node1 node2 
for link in links.findall('link'):
    for attributes in link.findall('attributes'):
        for attribute in attributes.findall('attribute'):
            if (attribute.get('name')=='nodes'):
                list_node = attribute.get('nodeId').split(' ')
                link.set('node1', list_node[0])
                link.set('node2',list_node[-1])

In [6]:
##set list width
for link in links.findall('link'):
    key = link.get('linkId')
    list_width = ""
    for k, v  in dict_hub[key].items():
        avg=0
        list_str = v.split(' ')
        for w in list_str:
            avg= avg+float(w)
        avg = avg/ len(list_str)  
        list_width = list_width +" "+ str(avg)
    for attributes in link.findall('attributes'):
        ET.SubElement(attributes, "attribute",{'Type':'Topology','name':'width','width':list_width})

In [7]:
## set osm 
osm_tree = ET.parse('centerline_osmid.xml')
osm_root = osm_tree.getroot()

for link in links.findall('link'):
    #add sub element list_osm
    for centerline in osm_root.findall('centerline'):
        if (centerline.get('centerline_id')== link.get('linkId')):
            #osmid
            for attributes in link.findall('attributes'):
                ET.SubElement(attributes, "attribute",{'Type':'Topology','name':'osm_way','way_id':centerline.get('osm_id')})


In [8]:
tree = ET.ElementTree(root)
ET.indent(tree)
tree.write('output.xml',encoding='utf-8')